In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#Remplacer les chemins d'accès
train = "plantnet_300K/images/images_train"    
test = "plantnet_300K/images/images_test"       
val = "plantnet_300K/images/images_val"  


batch_size = 16
img_height = 360
img_width = 360
epochs=10


train_ds = tf.keras.utils.image_dataset_from_directory(
  train,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  test,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names


#..............................................................

normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print("\nnp.min(first_image), np.max(first_image)", np.min(first_image), np.max(first_image))

#......................................................


num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()


#.........................................................................;


history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

model.save('model_classifiation')
#...........................................................;;

validation = tf.keras.utils.image_dataset_from_directory(
    val,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False)

val_predictions = model.predict(validation)

predi = []
for i in range(len(val_predictions)):
    predi.append(np.argmax(val_predictions[i]))  

val_labels = []
for images, labels in validation:
    batch_labels = tf.argmax(labels, axis=1)
    val_labels.extend(batch_labels.numpy())


val_accuracy = np.count_nonzero((np.array(predi) - np.array(val_labels)) == 0)/len(predi)
print('\nAccuracy on validation set:', val_accuracy)

2023-04-17 19:19:22.076127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 19:19:22.806615: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib:/raid/home/labuseraber/environment/testEnvironment/.conda9/lib/:/lib
2023-04-17 19:19:22.806717: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib:/raid/home/labuserabe

Found 45288 files belonging to 151 classes.


2023-04-17 19:19:27.563514: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 19:19:29.273093: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30969 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2023-04-17 19:19:29.275505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30969 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:07:00.0, compute capability: 7.0


Found 31112 files belonging to 1081 classes.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089

np.min(first_image), np.max(first_image) 0.0013921937 1.0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 360, 360, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 360, 360, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 180, 180, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 180, 180, 32)      4640      
               

2023-04-17 19:19:35.295564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-04-17 19:19:36.394410: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f5ea2c54ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-17 19:19:36.394442: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-04-17 19:19:36.394454: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-04-17 19:19:36.399730: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-17 19:19:36.558255: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2831/2831 [==============================] - 93s 31ms/step - loss: 2.4140 - accuracy: 0.3944 - val_loss: nan - val_accuracy: 0.0010
Epoch 2/10
2831/2831 [==============================] - 88s 31ms/step - loss: 1.5175 - accuracy: 0.6024 - val_loss: nan - val_accuracy: 0.0012
Epoch 3/10
2831/2831 [==============================] - 89s 31ms/step - loss: 0.6911 - accuracy: 0.8081 - val_loss: nan - val_accuracy: 0.0015
Epoch 4/10
2831/2831 [==============================] - 88s 31ms/step - loss: 0.2707 - accuracy: 0.9230 - val_loss: nan - val_accuracy: 0.0012
Epoch 5/10
2831/2831 [==============================] - 89s 31ms/step - loss: 0.1615 - accuracy: 0.9525 - val_loss: nan - val_accuracy: 0.0012
Epoch 6/10
2831/2831 [==============================] - 89s 31ms/step - loss: 0.1305 - accuracy: 0.9626 - val_loss: nan - val_accuracy: 0.0012
Epoch 7/10
2831/2831 [==============================] - 88s 31ms/step - loss: 0.1238 - accuracy: 0.9656 - val_loss: nan - val_accuracy: 0.0014
Epoch 8/10

INFO:tensorflow:Assets written to: model_classifiation/assets


INFO:tensorflow:Assets written to: model_classifiation/assets


Found 31118 files belonging to 1081 classes.
1945/1945 [==============================] - 26s 13ms/step

Accuracy on validation set: 0.001671058551320779
